# 타이타닉 생존자 데이터 셋

In [1]:
%pip install -q seaborn


[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [61]:
# 타이타닉 데이터셋 불러오기
# (pandas에 기본 내장된 예제 데이터셋)
import seaborn as sns
import pandas as pd

# titanic 데이터 DataFrame
titanic = sns.load_dataset('titanic')

# 처음 5개 행 보기
print("타이타닉 데이터 미리보기:")
print(titanic.head())

# 데이터셋 정보 확인
print("\n데이터셋 정보:")
print(titanic.info())

# 데이터 요약 통계
print("\n데이터 요약 통계:")
print(titanic.describe())

타이타닉 데이터 미리보기:
   survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
0         0       3    male  22.0      1      0   7.2500        S  Third   
1         1       1  female  38.0      1      0  71.2833        C  First   
2         1       3  female  26.0      0      0   7.9250        S  Third   
3         1       1  female  35.0      1      0  53.1000        S  First   
4         0       3    male  35.0      0      0   8.0500        S  Third   

     who  adult_male deck  embark_town alive  alone  
0    man        True  NaN  Southampton    no  False  
1  woman       False    C    Cherbourg   yes  False  
2  woman       False  NaN  Southampton   yes   True  
3  woman       False    C  Southampton   yes  False  
4    man        True  NaN  Southampton    no   True  

데이터셋 정보:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0 

In [52]:
# 실제 데이터가 온다면
# 1. col 별 결측치 합계 확인
titanic.isna().sum()

# 2. 승객 등급 등장 빈도 확인
print(titanic['class'].value_counts())

# 3. 성별 빈도
print(titanic['sex'].value_counts())

# 4. 생존 여부 확인
print(titanic['survived'].value_counts())

class
Third     491
First     216
Second    184
Name: count, dtype: int64
sex
male      577
female    314
Name: count, dtype: int64
survived
0    549
1    342
Name: count, dtype: int64


In [ ]:
# 5. 성별에 따른 생존율
titanic.groupby('sex')['survived'].mean()
titanic.groupby('sex').agg({'survived': ['mean']})  # 더 복잡한 통계 가능


# 6. 승객 등급에 따른 생존율
titanic.groupby('pclass')['survived'].mean()
titanic.groupby('pclass').agg({'survived': ['mean']})

pclass
1    0.629630
2    0.472826
3    0.242363
Name: survived, dtype: float64

In [79]:
# 7. 성별 & 승객 등급에 따른 생존율
# 아마 성별이랑 승객 등급을 같이 맞추는 컬럼을 하나 만들겠지? 아니네.
titanic.groupby(['sex', 'pclass'])['survived'].mean()

# 피벗테이블 만들기
titanic.pivot_table(
    values='survived',
    index='sex',
    columns='pclass',
    aggfunc='mean'
)

# 8. 나이 그룹별 생존율
    # 이건 실제 그룹핑을 해야 함.
        # 신규 컬럼을 만들어서
titanic['age_group'] = pd.cut(
    titanic['age'],
    bins=[0, 12, 18, 35, 60, 100, 200],# 초과 ~ 이하?
    labels=['아동', '청소년', '청년', '중장년', '노년', '초노년']
)

# MY 오답 : titanic.groupby('age_group')['survived'].count() # age gorup이라고 썼음.
titanic.groupby('age_group', observed=False)['survived'].mean()
# observed 옵션 : 카테고리가 있는데, 해당되는 내용이 없을 때(100 ~ 200) True면 데이터 있는 것만 보여줌.

# 성별과 나이까지 같이 보고 싶다.
titanic.groupby(['age_group','sex'], observed=False)['survived'].mean()

age_group  sex   
아동         female    0.593750
           male      0.567568
청소년        female    0.750000
           male      0.088235
청년         female    0.783333
           male      0.180672
중장년        female    0.771429
           male      0.192000
노년         female    1.000000
           male      0.105263
초노년        female         NaN
           male           NaN
Name: survived, dtype: float64

In [87]:
# 데이터 새로 불러오기
titanic = sns.load_dataset('titanic')

# 결측치 확인
missing = titanic.isna().sum()
# 결측값 있는 항목만 확인
missing[missing > 0]
# 결측 비율 보는 법
miss_p = round(titanic.isna().mean() * 100, 2)
miss_p[miss_p > 0]

age            19.87
embarked        0.22
deck           77.22
embark_town     0.22
dtype: float64

In [134]:
# 전략을 세우자. 어떻게 채울까?
# age : 중요한 정보. 함부로 채우면 안 됨. 그러면 나이가 없는 인간은 그냥 행을 지우자.
    # 쌤은 평균이란 중앙값으로 채우겠다고 ㅇㅇ
        # 수정안1: 성별, 객실의 평균을 추정해서 따라가기
            # 내 생각: 전체 나이 비율에 맞춰서 랜덤으로 때려넣기...?
# embarked & e.t : 굳이 필요 없는 내용임. 미입력값으로 채워
    # 쌤은 가장 많은 사람들이 탄 곳으로 채우겠다고 ㅇㅇ
# deck: 전부 다 없는 정보인데, 뭘로 채워야하지. 얘도 그냥 미입력으로 채워.
    # 22%를 가지고 77%를 추측하면 안 됨. 그냥 열을 삭제.
        # 그냥 열을 안 쓰면 되긴 하는데,,,,

# 한 가지 주의할 것.
    # 결측치를 채우기 전의 처음 값과 카피를 구분해두는 것이 필요함.
    # how to COPY

titanic = sns.load_dataset('titanic')

titanic_processed = titanic.copy()

# 비어있던 행 마스킹
age_mask = titanic['age'].isna()


# 나이 결측치 채우기
mean_age = titanic['age'].mean()
titanic_processed['age'] = titanic['age'].fillna(mean_age)


titanic_processed.isna().sum()

# 평균으로 채워진 값들만 확인
titanic_processed.loc[mask]


,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
5,0,3,male,29.699118,0,0,8.4583,Q,Third,man,True,NaN,Queenstown,no,True
17,1,2,male,29.699118,0,0,13.0000,S,Second,man,True,NaN,Southampton,yes,True
19,1,3,female,29.699118,0,0,7.2250,C,Third,woman,False,NaN,Cherbourg,yes,True
26,0,3,male,29.699118,0,0,7.2250,C,Third,man,True,NaN,Cherbourg,no,True
28,1,3,female,29.699118,0,0,7.8792,Q,Third,woman,False,NaN,Queenstown,yes,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,0,3,male,29.699118,0,0,7.2292,C,Third,man,True,NaN,Cherbourg,no,True
863,0,3,female,29.699118,8,2,69.5500,S,Third,woman,False,NaN,Southampton,no,False
868,0,3,male,29.699118,0,0,9.5000,S,Third,man,True,NaN,Southampton,no,True
878,0,3,male,29.699118,0,0,7.8958,S,Third,man,True,NaN,Southampton,no,True


In [119]:
# 일단 성별 & 객실 평균을 구하고
titanic.groupby(['sex', 'pclass'])['age'].mean()

# 평균은 구했는데, 알아서 맞춰주는 건...?
mean_ages = titanic.groupby(['sex', 'pclass'])['age'].transform('mean') # 결측치가 아니라 그냥 다 바뀜.
# 이런 규칙이 적용된 series를 만든 것.

# 특성에 맞는 평균값을 결측치에 넣어주는 작업.
titanic['age'].fillna(mean_ages) # NaN에 mean_ages시리즈를 채워라~

titanic_processed.loc[age_mask]
# titanic_processed.loc[titanic['age'].isna()]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
5,0,3,male,29.699118,0,0,8.4583,Q,Third,man,True,NaN,Queenstown,no,True
17,1,2,male,29.699118,0,0,13.0000,S,Second,man,True,NaN,Southampton,yes,True
19,1,3,female,29.699118,0,0,7.2250,C,Third,woman,False,NaN,Cherbourg,yes,True
26,0,3,male,29.699118,0,0,7.2250,C,Third,man,True,NaN,Cherbourg,no,True
28,1,3,female,29.699118,0,0,7.8792,Q,Third,woman,False,NaN,Queenstown,yes,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
859,0,3,male,29.699118,0,0,7.2292,C,Third,man,True,NaN,Cherbourg,no,True
863,0,3,female,29.699118,8,2,69.5500,S,Third,woman,False,NaN,Southampton,no,False
868,0,3,male,29.699118,0,0,9.5000,S,Third,man,True,NaN,Southampton,no,True
878,0,3,male,29.699118,0,0,7.8958,S,Third,man,True,NaN,Southampton,no,True


In [126]:
# embarked -> 최빈값으로 변경

# 비어있는 항구 개수
titanic['embarked'].isna().sum()

# 최빈값
mode_embarked = titanic['embarked'].mode()[0]   # Series에서 한 개 뽑기.

titanic_processed['embarked'] = titanic['embarked'].fillna(mode_embarked)

titanic_processed['embarked'].isna().sum()  # 2에서 0으로 바꼈다~

np.int64(0)

In [ ]:
# deck은 77%가 비어있으므로 해당 col을 삭제하겠음
titanic = sns.load_dataset('titanic')

titanic_processed = titanic.copy()

titanic['deck'].isna().mean() * 100     # 빈 값 비율

# # deck 컬럼 삭제
titanic_processed = titanic.drop(['deck', 'embarked'], axis=1)  # 리턴값이 있기 때문에 변수에 저장하지 않으면 날아감.
# inplace는 원본을 건드린다고,..ㅠㅠㅠ 헷갈려 슈밤


titanic_processed

,survived,pclass,sex,age,sibsp,parch,fare,class,who,adult_male,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,Third,man,True,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,First,woman,False,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,Third,woman,False,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,First,woman,False,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,Third,man,True,Southampton,no,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,Second,man,True,Southampton,no,True
887,1,1,female,19.0,0,0,30.0000,First,woman,False,Southampton,yes,True
888,0,3,female,NaN,1,2,23.4500,Third,woman,False,Southampton,no,False
889,1,1,male,26.0,0,0,30.0000,First,man,True,Cherbourg,yes,True
